In [ ]:
import pandas as pd
import numpy as np

import os
#os.getcwd()

# Testing Function 
imgDirTrain = r"C:\Users\lexei\Downloads\INFO 371\\"
imgDirVal = r"C:\Users\lexei\Downloads\INFO 371\\"
imgWidth, imgHeight = 64, 64
imgSize = (imgWidth, imgHeight)
## How many color channels
imgChannels = 3
# Creating Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,\
	Dropout, Flatten, Dense, Activation,\
	BatchNormalization, Input, Rescaling
from keras.utils import image_dataset_from_directory
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from PIL import Image


# sequential (not recursive) model (one input, one output)
# function that llows you to choose how many times you want to run the model to get an average accuracy
def modelTimes(imgTrain, imgVal, numTimesRun):
    imgDirTrain = imgTrain
    imgDirVal = imgVal
    accuracies = []
    for i in range(numTimesRun) :
        model = Sequential([
            Input(shape = (imgWidth, imgHeight, imgChannels)),
            #Rescaling(1./255),
            Conv2D(32, 3, activation='relu'),
            BatchNormalization(),
            MaxPooling2D(pool_size=2),
            Dropout(0.25),

            Conv2D(64, 3, activation='relu'),
            BatchNormalization(),
            MaxPooling2D(pool_size=2),
            Dropout(0.25),

           Conv2D(128, 3, activation='relu'),
           BatchNormalization(),
           MaxPooling2D(pool_size=2),
           Dropout(0.25),

            Flatten(),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.5),

            Dense(5, activation='softmax')
        ])

        model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        model.summary()

        imgSize = (64, 64)

        ## Dataset generators
        trainDS = image_dataset_from_directory(
            imgDirTrain + "train/",
            label_mode = "int",  # must match with 'sparse_categorical_crossentropy'
            color_mode = "rgb",  # color channels
            batch_size = 20, # WAS 32
            image_size = imgSize,
            shuffle = True,
            seed = 3)

        valDS = image_dataset_from_directory(
            imgDirVal + "validation/",
            label_mode = "int",  # must match with 'sparse_categorical_crossentropy'
            color_mode = "rgb",  # color channels
            batch_size = 20, # WAS 32
            image_size = imgSize,
            shuffle = False,
            seed = 3)

        print("Found classes:", trainDS.class_names)  # alphabetical order
        limitedTrainDS = trainDS

        yt = np.concatenate([y for x, y in limitedTrainDS], axis=0)
        count = pd.Series(yt).value_counts().sort_index()
        count.index = trainDS.class_names
        print("Class distribution in training set:" + str(count))

        print("Training")
        history = model.fit(
            limitedTrainDS,
            epochs = 150  # adjust for more
        )

        limitedValDS = valDS
        predictions = model.predict(limitedValDS)
        ## Validation
        yv = np.concatenate([y for x, y in limitedValDS], axis=0)
        print("Predicting")
        phat = model.predict(limitedValDS)
        print("Predicted array shape:", phat.shape)
        print("Example predicted probabilities:")
        print(phat[:5])

         ## Convert probabilities to categories:
        yhat = np.argmax(phat, axis=-1)

        print(f"Accuracy: {np.mean(yhat == yv):.3}")
        print(confusion_matrix(yhat, yv))
        accuracies.append(np.mean(yhat == yv))

        def print_labels(dataset, name):
            labels = np.concatenate([y.numpy() for x, y in dataset], axis=0)
            print(f"{name} labels: {np.unique(labels)}, counts: {np.bincount(labels)}")

        print_labels(trainDS, "Train")
        print_labels(valDS, "Validation")
        print(f"End of layer {i}")
        print(imgDirVal)

        # this is probably not the most convenitent way to do it but I pull from every folder
        valDirZN = r"C:\Users\lexei\Downloads\INFO 371\validation\ZN\\"
        valDirEN = r"C:\Users\lexei\Downloads\INFO 371\validation\EN\\"
        valDirRU = r"C:\Users\lexei\Downloads\INFO 371\validation\RU\\"
        valDirTH = r"C:\Users\lexei\Downloads\INFO 371\validation\TH\\"
        valDirDA = r"C:\Users\lexei\Downloads\INFO 371\validation\DA\\"

        # sets up the file paths so that I can randomly pick 20
        valDirs = [valDirDA, valDirZN, valDirEN, valDirRU, valDirTH]
        allImagePaths = []
        for valDir in valDirs:
            allImagePaths += [os.path.join(valDir, f) for f in os.listdir(valDir)]

        # randomly picks 20 image paths
        testFNames = list(np.random.choice(allImagePaths, size=20, replace=False))
        print("Plot example results")
        labelMap =  {0: "DA", 1: "EN", 2: "RU", 3: "TH", 4: "ZN"}

        plt.figure(figsize=(12, 24))
        index = 1
        rows, cols = 4, 5
        for testFName in testFNames:
            img = load_img(testFName, target_size=imgSize)
            im = Image.open(testFName).convert("RGB") # have to convert to RGB so the images can go back through the model
            im = im.resize(imgSize)
            im = np.array(im)
            im = np.expand_dims(im, axis=0)
            phat = model.predict(im)
            yhat = np.argmax(phat, axis=-1)[0]

            # picks the photos that the model isn't confident in classifying
            if (phat.max() < 0.9): 
                plt.subplot(rows, cols, index)
                plt.imshow(img)
                print("sample image {} [{}] is {}".format(testFName, phat, labelMap[yhat]))
                plt.xlabel(f"{labelMap[yhat]} ({phat.max():.3})")
                plt.tight_layout()
                plt.show()
            if index >= len(testFNames):
                break 
            index += 1

    avgAccuracy = sum(accuracies) / len(accuracies)
    print(f"Average Accuracy: {avgAccuracy}")
    print("I am done!")

    return avgAccuracy



modelTimes(imgDirTrain, imgDirTrain, 1)
